In [0]:
# url: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

# dataset

In [0]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 10.2MB/s]


In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [0]:
print(vars(train_data.examples[0]))

{'text': ['If', 'you', 'were', 'a', 'director', 'that', 'was', 'looking', 'to', 'cast', 'female', 'victims', 'for', 'a', 'slasher', 'movie', ',', 'then', 'surely', 'it', 'would', 'make', 'sense', 'to', 'add', 'a', 'couple', 'of', 'porn', 'stars', '?', 'It', "'s", 'not', 'as', 'if', 'they', "'re", 'inexperienced', 'in', 'front', 'of', 'the', 'camera', ',', 'they', 'have', 'no', 'qualms', 'with', 'the', 'requisite', 'nudity', 'and', 'how', 'many', 'unattractive', 'porn', 'queens', 'can', 'you', 'name', '?', 'Christian', 'Viel', 'obviously', 'recognized', 'the', 'potential', 'of', 'mixing', 'hardcore', 'actresses', 'with', 'hard', '-', 'gore', 'effects', 'and', 'so', 'he', 'cast', 'four', 'of', 'adult', 'cinema', "'s", 'sexiest', 'and', 'most', 'notorious', 'stars', '.', 'Jenna', 'Jameson', ',', 'Chasey', 'Lain', ',', 'Ginger', 'Lynn', 'Allen', 'and', 'Taylor', 'Hayes', 'all', 'turn', 'up', 'for', 'cameos', 'in', 'arguably', 'the', 'most', 'intriguing', 'slasher', 'flick', 'to', 'be', 're

In [0]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [0]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [0]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [0]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202828), (',', 192481), ('.', 165675), ('a', 109843), ('and', 109612), ('of', 100724), ('to', 93949), ('is', 76223), ('in', 61226), ('I', 54466), ('it', 54097), ('that', 49709), ('"', 44112), ("'s", 43662), ('this', 42434), ('-', 36898), ('/><br', 35853), ('was', 35433), ('as', 30355), ('with', 29784)]


In [0]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [0]:

print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7febadf88c80>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

# Model Building

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


# Model Training

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

# Evaluation

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 7s
	Train Loss: 0.694 | Train Acc: 50.11%
	 Val. Loss: 0.698 |  Val. Acc: 49.69%
Epoch: 02 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.62%
	 Val. Loss: 0.698 |  Val. Acc: 49.59%
Epoch: 03 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.98%
	 Val. Loss: 0.698 |  Val. Acc: 50.73%
Epoch: 04 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.70%
	 Val. Loss: 0.698 |  Val. Acc: 49.56%
Epoch: 05 | Epoch Time: 1m 6s
	Train Loss: 0.693 | Train Acc: 49.95%
	 Val. Loss: 0.698 |  Val. Acc: 50.64%
